<a href="https://colab.research.google.com/github/alexk2206/Data_Driven_Fantasy_Football/blob/dev/PULP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

In [14]:
file_path = 'https://raw.githubusercontent.com/alexk2206/Data_Driven_Fantasy_Football/refs/heads/main/FantasyPros_2025_Draft_ALL_Rankings.csv'
df = pd.read_csv(file_path, on_bad_lines='skip')

def standardize_column_names(df):
    def standardize(col):
        col_lower = col.lower()
        if "player" in col_lower:
            return "Player"
        elif "avg" in col_lower:
            return "AVG"
        else:
            return col
    df.columns = [standardize(col) for col in df.columns]
    return df

df = standardize_column_names(df)

k = 0.02
df['Value'] = 1000 * np.exp(-k * (df['AVG'] - 1))
df['POS'] = df['POS'].str.replace('\d+', '', regex=True)
df

,RK,TIERS,Player,TEAM,POS,BEST,WORST,AVG,STD.DEV,ECR VS. ADP,Value
0,1,1,Ja'Marr Chase,CIN,WR,1,1,1.0,0.0,-,1000.000000
1,2,1,Justin Jefferson,MIN,WR,3,6,3.3,0.7,-,955.041962
2,3,1,Bijan Robinson,ATL,RB,2,5,3.8,1.5,-,945.539136
3,4,1,CeeDee Lamb,DAL,WR,3,7,4.2,0.9,-,938.005000
4,5,2,Saquon Barkley,PHI,RB,2,16,4.5,3.8,-,932.393820
...,...,...,...,...,...,...,...,...,...,...,...
403,404,16,Elijah Higgins,ARI,TE,324,375,349.5,25.5,-,0.939653
404,405,16,Malik Willis,GB,QB,321,386,353.5,32.5,-,0.867409
405,406,16,Luke Schoonmaker,DAL,TE,352,373,362.5,10.5,-,0.724521
406,407,16,Grant Calcaterra,PHI,TE,353,380,366.5,13.5,-,0.668817


In [16]:
df = df[['Player', 'POS', 'Value']].dropna()

# Only keep relevant positions
df = df[df['POS'].str.contains('QB|RB|WR|TE|K|DST', na=False)]

df_small = df.nlargest(220, 'Value').copy()

print(df_small['Player'].duplicated().sum())
print(df_small['POS'].value_counts())
print(df_small.head(20))

0
POS
WR     78
RB     61
QB     28
TE     26
DST    18
K       9
Name: count, dtype: int64
                 Player POS        Value
0         Ja'Marr Chase  WR  1000.000000
1      Justin Jefferson  WR   955.041962
2        Bijan Robinson  RB   945.539136
3           CeeDee Lamb  WR   938.005000
4        Saquon Barkley  RB   932.393820
5            Puka Nacua  WR   894.044258
6     Amon-Ra St. Brown  WR   876.340995
7          Jahmyr Gibbs  RB   874.590065
8          Malik Nabers  WR   860.707976
9          Nico Collins  WR   836.942423
10     Brian Thomas Jr.  WR   831.935804
11         Drake London  WR   771.051586
12        De'Von Achane  RB   761.854261
13         Brock Bowers  TE   755.783741
14           A.J. Brown  WR   749.761592
15   Jaxon Smith-Njigba  WR   723.250242
16  Christian McCaffrey  RB   707.512487
17        Ladd McConkey  WR   700.472620
18         Trey McBride  TE   670.320046
19        Derrick Henry  RB   663.650250


In [4]:
!pip install pulp
from pulp import LpProblem, LpMaximize, LpVariable, lpSum, LpBinary, PULP_CBC_CMD

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 94.3 MB/s eta 0:00:00


# MIP, no Draft order

In [6]:
number_of_teams = 8
teams = list(range(1, number_of_teams + 1))
min_pos = {'QB': 1, 'RB': 2, 'WR': 2, 'TE': 1, 'K': 1, 'DST': 1}
max_team_size = 15

players = df.index.tolist()

# Modell erstellen
model = LpProblem("Fantasy_Football_Draft", LpMaximize)

# Entscheidungsvariablen x[t,i]
x = LpVariable.dicts("x", ((t, i) for t in teams for i in players), cat=LpBinary)

# Zielfunktion: Maximiere den Gesamtwert aller Teams
model += lpSum(df.loc[i, "Value"] * x[t, i] for t in teams for i in players)

# 1. Jeder Spieler kann nur von einem Team ausgewählt werden
for i in players:
    model += lpSum(x[t, i] for t in teams) <= 1

# 2. Teamgröße: maximal 15 Spieler pro Team
for t in teams:
    model += lpSum(x[t, i] for i in players) <= max_team_size

# 3. Mindestanzahl pro Position pro Team
for t in teams:
    for pos, min_count in min_pos.items():
        model += lpSum(x[t, i] for i in players if df.loc[i, "POS"] == pos) >= min_count

# Modell lösen
solver = PULP_CBC_CMD(msg=0)
model.solve(solver)

# Ergebnis ausgeben
result = {}
for t in teams:
    team_players = [i for i in players if x[t, i].varValue == 1]
    result[t] = df.loc[team_players, ["Player", "POS", "Value"]].reset_index(drop=True)

# Ausgabe: result ist ein Dictionary mit Teamnummer als Key und DataFrame als Value
for t in teams:
    print(f"\nTeam {t}:")
    print(result[t])



Team 1:
                 Player  POS       Value
0          Travis Kelce   TE  616.313202
1         Rachaad White   RB  542.265253
2   Michael Pittman Jr.   WR  510.686183
3             Joe Mixon   RB  471.422764
4          Alvin Kamara   RB  462.087968
5            DK Metcalf   WR  447.535238
6           Zamir White   RB  269.280956
7        Jayden Daniels   QB  131.073113
8          Keon Coleman   WR  125.933662
9      Brian Thomas Jr.   WR   99.858609
10       Evan McPherson    K   51.612056
11       Deshaun Watson   QB   38.542259
12       Miami Dolphins  DST   32.975184
13       Xavier Legette   WR   28.667248
14       MarShawn Lloyd   RB   26.782676

Team 2:
                Player  POS       Value
0   Travis Etienne Jr.   RB  720.363020
1         Nico Collins   WR  580.421915
2      Diontae Johnson   WR  174.819721
3   Brian Robinson Jr.   RB  160.093061
4          Chase Brown   RB  110.360831
5      Trevor Lawrence   QB   76.688770
6        Khalil Shakir   WR   72.802863
7     

# MIP, Snake Draft

In [7]:
# Parameter
number_of_teams = 4
max_team_size = 15
min_pos = {'QB': 1, 'RB': 2, 'WR': 2, 'TE': 1, 'K': 1, 'DST': 1}

teams = list(range(1, number_of_teams + 1))
players = df.index.tolist()
num_picks = max_team_size * number_of_teams

# Snake-Draft-Reihenfolge erzeugen
draft_order = []
for r in range(max_team_size):
    if r % 2 == 0:
        draft_order += teams
    else:
        draft_order += teams[::-1]
# draft_order[p] = Team, das bei Pick p dran ist

# MIP-Modell
model = LpProblem("Fantasy_Snake_Draft", LpMaximize)

# Entscheidungsvariablen: x[t, i, p] = 1, wenn Team t Spieler i bei Pick p wählt
x = LpVariable.dicts(
    "x",
    ((t, i, p) for t in teams for i in players for p in range(num_picks)),
    cat=LpBinary
)

# Zielfunktion: Summe der Werte aller gezogenen Spieler maximieren
model += lpSum(
    df.loc[i, "Value"] * x[t, i, p]
    for t in teams for i in players for p in range(num_picks)
)

# 1. Jeder Pick wird von genau einem Team gemacht, und das nur in der Draft-Reihenfolge
for p in range(num_picks):
    t = draft_order[p]
    model += lpSum(x[t, i, p] for i in players) == 1
    # Alle anderen Teams dürfen bei diesem Pick niemanden wählen
    for t2 in teams:
        if t2 != t:
            model += lpSum(x[t2, i, p] for i in players) == 0

# 2. Jeder Spieler wird höchstens einmal gewählt
for i in players:
    model += lpSum(x[t, i, p] for t in teams for p in range(num_picks)) <= 1

# 3. Jedes Team wählt genau max_team_size Spieler
for t in teams:
    model += lpSum(x[t, i, p] for i in players for p in range(num_picks)) == max_team_size

# 4. Mindestanzahl pro Position pro Team
for t in teams:
    for pos, min_count in min_pos.items():
        model += lpSum(
            x[t, i, p]
            for i in players if df.loc[i, "POS"] == pos
            for p in range(num_picks)
        ) >= min_count

# Modell lösen
solver = PULP_CBC_CMD(msg=1)
model.solve(solver)

# Ergebnisse sammeln: Für jeden Pick, wer wurde von wem gezogen?
draft_results = []
for p, t in enumerate(draft_order):
    for i in players:
        if x[t, i, p].varValue == 1:
            draft_results.append({
                "Pick": p + 1,
                "Team": t,
                "Player": df.loc[i, "Player"],
                "POS": df.loc[i, "POS"],
                "Value": df.loc[i, "Value"]
            })

draft_df = pd.DataFrame(draft_results)
print(draft_df)

# Optional: Team-Zusammenfassungen ausgeben
for t in teams:
    team_picks = draft_df[draft_df["Team"] == t]
    print(f"\nTeam {t}:")
    print(team_picks[["Pick", "Player", "POS", "Value"]].reset_index(drop=True))


    Pick  Team               Player  POS        Value
0      1     1   Travis Etienne Jr.   RB   720.363020
1      2     2       Saquon Barkley   RB   848.742022
2      3     3          CeeDee Lamb   WR   968.506582
3      4     4           DK Metcalf   WR   447.535238
4      5     4         Younghoe Koo    K    62.038507
5      6     3      Harrison Butker    K   108.175540
6      7     2          Jalen Hurts   QB   546.620774
7      8     1         Alvin Kamara   RB   462.087968
8      9     1          Josh Jacobs   RB   580.421915
9     10     2     Deebo Samuel Sr.   WR   537.944438
10    11     3  Christian McCaffrey   RB  1000.000000
11    12     4   Patrick Mahomes II   QB   571.209064
12    13     4        Ja'Marr Chase   WR   894.044258
13    14     3          Tyreek Hill   WR   956.953957
14    15     2         Jahmyr Gibbs   RB   789.780674
15    16     1     Justin Jefferson   WR   886.920437
16    17     1          Breece Hall   RB   915.760877
17    18     2         Mark 

# MIP: Greedy Snake Draft

In [8]:
number_of_teams = 4
max_team_size = 15
min_pos = {'QB': 1, 'RB': 2, 'WR': 2, 'TE': 1, 'K': 1, 'DST': 1}

teams = list(range(1, number_of_teams + 1))
rosters = {t: [] for t in teams}
roster_counts = {t: {pos: 0 for pos in min_pos} for t in teams}

# Snake-Draft-Reihenfolge
draft_order = []
for r in range(max_team_size):
    if r % 2 == 0:
        draft_order += teams
    else:
        draft_order += teams[::-1]

available = df.copy()
draft_results = []

for pick, t in enumerate(draft_order):
    # Team-Needs bestimmen
    needs = []
    for pos, min_c in min_pos.items():
        if roster_counts[t][pos] < min_c:
            needs.append(pos)
    # Wenn alle Mindestanforderungen erfüllt, alle Positionen zulassen
    if not needs:
        needs = list(min_pos.keys())
    # Wähle den besten verfügbaren Spieler, der eine Need-Position abdeckt
    candidates = available[available["POS"].isin(needs)]
    if candidates.empty:
        candidates = available  # Falls keine Need mehr, beliebiger Spieler
    best_idx = candidates["Value"].idxmax()
    best_player = available.loc[best_idx]
    # Roster aktualisieren
    rosters[t].append(best_player["Player"])
    roster_counts[t][best_player["POS"]] += 1
    draft_results.append({
        "Pick": pick + 1,
        "Team": t,
        "Player": best_player["Player"],
        "POS": best_player["POS"],
        "Value": best_player["Value"]
    })
    available = available.drop(best_idx)
    # Team voll? Dann überspringen in Zukunft (optional)

draft_df = pd.DataFrame(draft_results)
print(draft_df)


    Pick  Team               Player  POS        Value
0      1     1  Christian McCaffrey   RB  1000.000000
1      2     2          CeeDee Lamb   WR   968.506582
2      3     3          Tyreek Hill   WR   956.953957
3      4     4       Bijan Robinson   RB   923.116346
4      5     4          Breece Hall   RB   915.760877
5      6     3    Amon-Ra St. Brown   WR   901.225297
6      7     2        Ja'Marr Chase   WR   894.044258
7      8     1     Justin Jefferson   WR   886.920437
8      9     1       Saquon Barkley   RB   848.742022
9     10     2      Jonathan Taylor   RB   828.614707
10    11     3         Jahmyr Gibbs   RB   789.780674
11    12     4           A.J. Brown   WR   831.935804
12    13     4       Garrett Wilson   WR   796.124260
13    14     3       Kyren Williams   RB   740.818221
14    15     2   Travis Etienne Jr.   RB   720.363020
15    16     1           Puka Nacua   WR   767.973540
16    17     1           Josh Allen   QB   628.763554
17    18     2         Travi

# MIP: B. Optional: Value Over Replacement (VOR)

In [17]:
number_of_teams = 8
roster_requirements = {'QB': 1, 'RB': 2, 'WR': 2, 'TE': 1, 'K': 1, 'DST': 1}
max_team_size = 15

# Schritt 1: Replacement-Level pro Position bestimmen
replacement_rank = {pos: number_of_teams * req for pos, req in roster_requirements.items()}

replacement_value = {}
for pos in roster_requirements:
    pos_players = df[df['POS'] == pos].sort_values(by="Value", ascending=False).reset_index(drop=True)
    idx = replacement_rank[pos] - 1  # Nullbasiert
    if idx < len(pos_players):
        replacement_value[pos] = pos_players.loc[idx, "Value"]
    else:
        replacement_value[pos] = pos_players["Value"].min()

# Schritt 2: VOR berechnen
df["VOR"] = df.apply(lambda row: row["Value"] - replacement_value[row["POS"]], axis=1)

# Schritt 3: Snake-Draft nach VOR
teams = list(range(1, number_of_teams + 1))
rosters = {t: [] for t in teams}
roster_counts = {t: {pos: 0 for pos in roster_requirements} for t in teams}

draft_order = []
for r in range(max_team_size):
    if r % 2 == 0:
        draft_order += teams
    else:
        draft_order += teams[::-1]

available = df.copy()
draft_results = []

for pick, t in enumerate(draft_order):
    needs = []
    for pos, min_c in roster_requirements.items():
        if roster_counts[t][pos] < min_c:
            needs.append(pos)
    if not needs:
        needs = list(roster_requirements.keys())
    candidates = available[available["POS"].isin(needs)]
    if candidates.empty:
        candidates = available
    best_idx = candidates["VOR"].idxmax()
    best_player = available.loc[best_idx]
    # Roster aktualisieren (jetzt als Dict, nicht nur Name)
    rosters[t].append({
        "Player": best_player["Player"],
        "POS": best_player["POS"],
        "Value": best_player["Value"],
        "VOR": best_player["VOR"]
    })
    roster_counts[t][best_player["POS"]] += 1
    draft_results.append({
        "Pick": pick + 1,
        "Team": t,
        "Player": best_player["Player"],
        "POS": best_player["POS"],
        "Value": best_player["Value"],
        "VOR": best_player["VOR"]
    })
    available = available.drop(best_idx)

draft_df = pd.DataFrame(draft_results)
print("Draft Board:")
print(draft_df)

# Ausgabe: Spielerlisten pro Team
print("\nSpielerlisten pro Team:")
for t in teams:
    print(f"\nTeam {t}:")
    team_df = pd.DataFrame(rosters[t])
    print(team_df.reset_index(drop=True))


Draft Board:
     Pick  Team            Player POS       Value         VOR
0       1     1      Brock Bowers  TE  755.783741  569.782141
1       2     2    Bijan Robinson  RB  945.539136  542.209058
2       3     3    Saquon Barkley  RB  932.393820  529.063742
3       4     4      Trey McBride  TE  670.320046  484.318445
4       5     5      Jahmyr Gibbs  RB  874.590065  471.259987
..    ...   ...               ...  ..         ...         ...
115   116     4       Brock Purdy  QB  147.194564  -78.629287
116   117     5    Dalton Kincaid  TE  105.188637  -80.812964
117   118     6    Caleb Williams  QB  133.453790  -92.370061
118   119     7       Jordan Love  QB  128.220992  -97.602859
119   120     8  David Montgomery  RB  304.830315  -98.499763

[120 rows x 6 columns]

Spielerlisten pro Team:

Team 1:
                Player  POS       Value         VOR
0         Brock Bowers   TE  755.783741  569.782141
1         Malik Nabers   WR  860.707976  292.915906
2         Nico Collins   WR  

# MIP: Combination Value and VOR

In [20]:
number_of_teams = 12
roster_requirements = {'QB': 1, 'RB': 2, 'WR': 2, 'TE': 1, 'K': 1, 'DST': 1}
max_team_size = 15

teams = list(range(1, number_of_teams + 1))
rosters = {t: [] for t in teams}
roster_counts = {t: {pos: 0 for pos in roster_requirements} for t in teams}

draft_order = []
for r in range(max_team_size):
    if r % 2 == 0:
        draft_order += teams
    else:
        draft_order += teams[::-1]

# Neue Spalte für den kombinierten Score
Value_portion = 0.8
VOR_portion = 1 - Value_portion
df["CombinedScore"] = Value_portion * df["Value"] + VOR_portion * df["VOR"]

available = df.copy()
draft_results = []

for pick, t in enumerate(draft_order):
    needs = []
    for pos, min_c in roster_requirements.items():
        if roster_counts[t][pos] < min_c:
            needs.append(pos)
    if not needs:
        needs = list(roster_requirements.keys())
    candidates = available[available["POS"].isin(needs)]
    if candidates.empty:
        candidates = available
    best_idx = candidates["CombinedScore"].idxmax()
    best_player = available.loc[best_idx]
    rosters[t].append({
        "Player": best_player["Player"],
        "POS": best_player["POS"],
        "Value": best_player["Value"],
        "VOR": best_player["VOR"],
        "CombinedScore": best_player["CombinedScore"]
    })
    roster_counts[t][best_player["POS"]] += 1
    draft_results.append({
        "Pick": pick + 1,
        "Team": t,
        "Player": best_player["Player"],
        "POS": best_player["POS"],
        "Value": best_player["Value"],
        "VOR": best_player["VOR"],
        "CombinedScore": best_player["CombinedScore"]
    })
    available = available.drop(best_idx)

draft_df = pd.DataFrame(draft_results)

# Spielerlisten pro Team ausgeben
for t in teams:
    print(f"\nTeam {t}:")
    team_df = pd.DataFrame(rosters[t])
    print(team_df.reset_index(drop=True))


Team 1:
            Player  POS        Value         VOR  CombinedScore
0    Ja'Marr Chase   WR  1000.000000  432.207929     886.441586
1    Ashton Jeanty   RB   621.263482  217.933404     540.597467
2      Tyreek Hill   WR   645.325783   77.533712     531.767369
3        Joe Mixon   RB   397.722800   -5.607279     317.056784
4   Baker Mayfield   QB   301.797203   75.973351     256.632433
5     Tucker Kraft   TE   120.031629  -65.969972      82.831308
6   Denver Broncos  DST    50.186962   25.709439      45.291458
7       Tyler Bass    K    11.493197   -3.293905       8.535777
8    DeVonta Smith   WR   374.561227 -193.230844     261.002813
9      C.J. Stroud   QB   116.019152 -109.804700      70.854381
10   Jakobi Meyers   WR   184.150854 -383.641217      70.592440
11   J.J. McCarthy   QB    62.411857 -163.411995      17.247087
12   Rachaad White   RB    97.881276 -305.448802      17.215261
13   Will Reichard    K     7.536480   -7.250621       4.579060
14    Joshua Karty    K     7.0

In [19]:
draft_df

,Pick,Team,Player,POS,Value,VOR,CombinedScore
0,1,1,Ja'Marr Chase,WR,1000.000000,432.207929,886.441586
1,2,2,Bijan Robinson,RB,945.539136,542.209058,864.873120
2,3,3,Saquon Barkley,RB,932.393820,529.063742,851.727804
3,4,4,Justin Jefferson,WR,955.041962,387.249891,841.483548
4,5,5,CeeDee Lamb,WR,938.005000,370.212929,824.446585
...,...,...,...,...,...,...,...
115,116,4,Justin Fields,QB,107.099175,-118.724676,61.934405
116,117,5,Trevor Lawrence,QB,105.610234,-120.213618,60.445464
117,118,6,Rhamondre Stevenson,RB,140.015801,-263.314277,59.349785
118,119,7,Quinshon Judkins,RB,139.178221,-264.151857,58.512206
